# Graphs

## Setup

In [ ]:
import sys
sys.path.append('../../src/')

from utils.bot import Bot
from configparser import ConfigParser
import plotly.express as px
from pathlib import Path
import pandas as pd

In [ ]:
config_file = Path('../../src/config.ini')
assert config_file.exists(), f'{config_file} doesn\'t exist.'
config = ConfigParser()
config.read(config_file)

insta_u, insta_p = config['INSTAGRAM']['KONT_USERNAME'], config['INSTAGRAM']['KONT_PASSWORD']
google_key_file = '../../src/keys/kontstats-d1e621a98972.json'
google_api_key = config['GOOGLE']['API_KEY']
spotify_client_id = config['SPOTIFY']['CLIENT_ID']
spotify_client_secret = config['SPOTIFY']['CLIENT_SECRET']
bot_deats = [insta_u, insta_p,
                google_key_file,
                spotify_client_id,
                spotify_client_secret,
                google_api_key]

## Get data

In [ ]:
client = Bot(*bot_deats)
df = client.GSpread.get_raw_logs_as_df()
df.head()

## Prep Data

In [ ]:
def clean_data(df):
    """
    Function that cleans the data from the sheet to a clean format
    """
    df.DATETIME = pd.to_datetime(df.DATETIME, format='%d/%m/%Y %H:%M:%S')
    #Add day for aggregations
    df['DATETIME_D'] = df.DATETIME.dt.round('D')
    
    spotify_df = df[(df.PLATFORM == 'SPOTIFY') & (df.SONG != '')]
    youtube_df = df[(df.PLATFORM == 'YOUTUBE') & (df.SONG != '')]
    apple_df = df[(df.PLATFORM == 'APPLE') & (df.SONG != '')]
    instagram_df = df[df.PLATFORM == 'INSTAGRAM']
    
    return df, spotify_df, youtube_df, apple_df, instagram_df

In [ ]:
df, spotify_df, youtube_df, apple_df, instagram_df = clean_data(df)

## Graphs

### Daily streams per platform

In [ ]:
def get_daily_plays_per_platform(df):
    daily_plays = df[df.PLATFORM.isin(['SPOTIFY', 'YOUTUBE', 'APPLE'])]
    daily_plays.MESSAGE = daily_plays.MESSAGE.apply(lambda x: 'PLAYS' if x == 'VIEWCOUNT' else x)
    daily_plays = daily_plays[daily_plays.MESSAGE == 'PLAYS']
    daily_plays = daily_plays.groupby(['DATETIME_D', 'PLATFORM', 'SONG']).max().VALUE.\
    reset_index().drop_duplicates()
    daily_plays.VALUE = daily_plays.VALUE.astype(int)
    daily_plays = daily_plays.groupby(['DATETIME_D', 'PLATFORM']).sum().VALUE.reset_index()
    return daily_plays

In [ ]:
daily_plays = get_daily_plays_per_platform(df)

In [ ]:
def plot_daily_plays_per_platform(df):
    fig = px.scatter(daily_plays,
                 x='DATETIME_D',
                 y='VALUE',
                 color_discrete_map={'SPOTIFY':'#66d36d',
                                    'YOUTUBE':'#eb3323',
                                    'APPLE':'#dddddd'},
                 color='PLATFORM',
                 title='Total Streams per Platform',
                 template='plotly_dark').update_traces(mode='lines+markers')
    return fig

In [ ]:
fig = plot_daily_plays_per_platform(daily_plays)
fig

### Total Plays Per Day

In [ ]:
def get_daily_plays(df):
    daily_plays = get_daily_plays_per_platform(df)
    return daily_plays.groupby('DATETIME_D').sum().VALUE.reset_index()

In [ ]:
fig = px.scatter(get_daily_plays(df),
                 x='DATETIME_D',
                 y='VALUE',
                 title='Total Streams',
                 template='plotly_dark').update_traces(mode='lines+markers')
fig

### Spotify

In [ ]:
spotify_df.head()

In [ ]:
fig = px.scatter(spotify_df,
                 x='DATETIME',
                 y='VALUE',
                 color='SONG',
                 title='Spotify song streams',
                 template='plotly_dark').update_traces(mode='lines+markers')
fig

### Youtube

In [ ]:
youtube_df.head()

In [ ]:
fig = px.scatter_3d(youtube_df.sort_values(by='DATETIME'),
                 x='DATETIME',
                 y='MESSAGE',
                 z='VALUE',
                 color='SONG',
                 log_z=True,
                 title=f'Youtube Stats',
                 template='plotly_dark')#.update_traces(mode='lines+markers')
fig

### Instagram

In [ ]:
instagram_df.head()

In [ ]:
fig = px.scatter(instagram_df[instagram_df.MESSAGE == 'FOLLOWER_COUNT'],
                 x='DATETIME',
                 y='VALUE',
                 color_discrete_sequence=['#f3a55c'],
                 title='Instagram Followers',
                 template='plotly_dark').update_traces(mode='lines+markers')
fig

### Plays on all platforms

In [ ]:
plays_per_day_df = df[df.PLATFORM.isin(['YOUTUBE', 'SPOTIFY']) &
             (df.SONG != '') & 
             (df.MESSAGE.isin(['PLAYS', 'VIEWCOUNT']))].\
drop('DATETIME', axis=1).drop_duplicates()
plays_per_day_df.VALUE = plays_per_day_df.VALUE.astype(float)
plays_per_day_df = plays_per_day_df.groupby(['SONG', 'PLATFORM','DATETIME_D']).max().reset_index()

plays_per_day_df.head()

In [ ]:
fig = px.scatter(plays_per_day_df.groupby(['DATETIME_D', 'SONG']).sum().reset_index(),
                 x='DATETIME_D',
                 y='VALUE',
                 color='SONG',
                 title='Total Streams per song',
                 template='plotly_dark').update_traces(mode='lines+markers')
fig

In [ ]:
px.scatter(plays_per_day_df.sort_values(by='DATETIME_D'),
              x='DATETIME_D',
              y='VALUE',
               facet_row='SONG',
               height=2000,
              color='PLATFORM',
              template='plotly_dark').update_traces(mode='lines+markers')